<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-connectivity" data-toc-modified-id="Test-connectivity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test connectivity</a></span></li><li><span><a href="#Test-watching-legimens.Object" data-toc-modified-id="Test-watching-legimens.Object-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test watching legimens.Object</a></span></li><li><span><a href="#Changing-sys.switchinterval" data-toc-modified-id="Changing-sys.switchinterval-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Changing sys.switchinterval</a></span></li><li><span><a href="#Simple-object-watch" data-toc-modified-id="Simple-object-watch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Simple object watch</a></span></li><li><span><a href="#Prettified-API" data-toc-modified-id="Prettified-API-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Prettified API</a></span></li></ul></div>

In [1]:
from libvis import Vis
import libvis
import json
import numpy as np

from legimens.Object import ref
from legimens import Object

%load_ext autoreload
%autoreload 2

No module named 'bokeh'


In [2]:
vis = Vis(ws_port=7700, vis_port=7000)
vis.start()

Starting http at 7000
2020-05-17 21:03:43.180 | INFO     | legimens.websocket.server:start_server:33 - Starting ws server at localhost:7700


## Test connectivity

In [3]:
vis.vars.image = 1


## Test watching legimens.Object


In [4]:

class Var(libvis.modules.BaseModule):
    def _prepare_send(self, name, value):
        return name, {'type':'Object','value':value}

x = Var(x=1)
vis.app.watch_obj(x)
vis.vars.x = x

2020-05-15 02:02:30.758 | INFO     | legimens.App:_handler:74 - New ws connection of /Legi_0x7f30399864a0 from ::1
2020-05-15 02:02:30.765 | INFO     | legimens.App:_handle_obj_ref:105 - Yield initial update {"x": {"type": "Object", "value": 1}}


In [5]:
for i in range(100000000//3):
    x['x'] = i

In [ ]:
vis.app._watch_poll_delay = .1
type(vis.app.vars)

## Changing sys.switchinterval 
determines how often we switch between threads to send updates

Try to set larger value and run previous tests

In [ ]:
import sys
sys.getswitchinterval()

In [ ]:
sys.setswitchinterval(0.04)

In [ ]:
vis.vars.foo=2

## Simple object watch

Note that CPython has a limitation in it's implementation of mutable objects:
it can't create a weakref for base types.
This is fixed for mutables in those additional classes

For more info check: https://mail.python.org/pipermail/python-list/2005-March/346298.html

In [6]:
class List(list):
    name='VisVar'
    pass

libvis.interface.add_serializer(List, ref)
def ser_type(value):
    return json.dumps({'value':value})
    return str(value)
vis.app.serialize_value = ser_type

li = List()



In [7]:
vis.app.watch_obj(li)
vis.vars.x = li

2020-05-15 02:02:41.257 | ERROR    | legimens.App:_handler:89 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1000, name=NORMAL_CLOSURE, reason=None>
2020-05-15 02:02:41.267 | INFO     | legimens.App:_handler:74 - New ws connection of /Legi_0x7f303b29c9a0 from ::1
2020-05-15 02:02:41.269 | INFO     | legimens.App:_handle_obj_ref:105 - Yield initial update {"value": []}


In [12]:
li.append(len(li))

In [15]:
li.pop()

2

2020-05-15 13:34:34.759 | ERROR    | legimens.App:_handler:89 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-15 13:34:34.766 | ERROR    | legimens.App:_handler:89 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>


Values should be instantly updated in x variable

## Prettified API

In [12]:
import time

In [4]:
x = [10]

vis.watch(x, 'x')

'Legi_0x7fabf2de57c0'

In [37]:
for i in range(100):
    x.append(i)
    time.sleep(.1)

2020-05-17 22:01:04.734 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:01:04.736 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:01:04.738 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:01:04.739 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:01:06.258 | INFO     | legimens.App:_handler:76 - New ws connection of / from ::1
2020-05-17 22:01:06.259 | INFO     | legimens.App:_handler:88 - Sending root Legi_0x7fabf2e0aae0
2020-05-17 22:01:06.295 | INFO     | legimens.App:_handler:76 - New ws connection of /u

In [38]:
for _ in range(100):
    x.pop()

2020-05-17 22:33:28.593 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:33:28.604 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1001, name=GOING_AWAY, reason=None>
2020-05-17 22:33:29.582 | INFO     | legimens.App:_handler:76 - New ws connection of / from ::1
2020-05-17 22:33:29.583 | INFO     | legimens.App:_handler:88 - Sending root Legi_0x7fabf2e0aae0
2020-05-17 22:33:29.610 | INFO     | legimens.App:_handler:76 - New ws connection of /undefined from ::1
2020-05-17 22:33:29.620 | ERROR    | legimens.App:_handle_obj_ref:98 - No child with ref undefined
2020-05-17 22:33:29.953 | INFO     | legimens.App:_handler:76 - New ws connection of /Legi_0x7fabf2e0aae0 from ::1
2020-05-17 22:33:29.954 | INFO     | legimens.App:_handle_obj_ref:107 - Yield initial update {"image": {"value": 1, "type": "raw

In [22]:
list(vis.app._child_obj.keys())
vis.vars
vis.app._watch_poll_delay = .2

In [39]:
vis.stop()

2020-05-17 22:38:21.035 | INFO     | legimens.App:_start:197 - App crashed.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/site-packages/trio/_core/_run.py", line 1804, in run
    raise runner.main_task_outcome.error
  File "/home/dali/side-projects-hobby/legimens/python/legimens/App.py", line 194, in _start
    nursery.start_soon(self._poll_objects)
  File "/usr/lib/python3.8/site-packages/trio/_core/_run.py", line 730, in __aexit__
    raise combined_error_from_nursery
  File "/home/dali/side-projects-hobby/legimens/python/legimens/App.py", line 162, in _monitor_updates
    message = json.dumps(updates)
  File "/usr/lib/python3.8/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/usr/lib/python3.8/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "

Stopping app server
Stopping websocket server
